In [ ]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPEN_AI_KEY")  # Replace with your actual OpenAI API key
client = OpenAI(api_key=api_key)

project_agent_creation_agent_id = "asst_mhrbxqgPluw4ca0fdlj7LnME"

In [ ]:
with open("project_outline.md", "r") as file:
    project_text = file.read()

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": f"Here is a project outline:\n {project_text}",
    }
  ]
)
print(thread.id)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=project_agent_creation_agent_id
)

In [ ]:
run_status = client.beta.threads.runs.retrieve(run_id=run.id, thread_id=thread.id,)
print(run_status.status)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(messages)

In [ ]:
message_str = messages.data[0].content[0].text.value

In [ ]:
import re

def save_code_from_markdown(filename, markdown_content) -> str:
    # Regular expression to find code blocks and the specified language
    code_block_pattern = re.compile(r"```(\w+?)\n(.*?)```", re.DOTALL)

    # Find all code blocks
    code_blocks = code_block_pattern.findall(markdown_content)

    if not code_blocks:
        print("No code blocks found.")
        return

    for index, (language, code) in enumerate(code_blocks):
        # Map language to file extensions (you might need to expand this dictionary)
        file_extension_map = {
            "python": ".py",
            "javascript": ".js",
            "html": ".html",
            "css": ".css",
            "json": ".json",
            
        }

        # Determine the file extension
        file_extension = file_extension_map.get(language.lower(), ".txt")

        # Create a file name (you might want to customize this)
        file_name = f"{filename}_{index}{file_extension}"

        # Save the file
        with open(file_name, "w") as file:
            file.write(code)
            print(f"Saved {file_name}")
        
        return file_name


In [ ]:
filename = save_code_from_markdown("agents", message_str)
with open(filename, "r") as file:
    agent_schema = json.loads(file.read())

In [ ]:
print(agent_schema)

In [ ]:
import time
## The agent_schema is an array of objects with name and description, loop through them
function_schema_agent = "asst_8OE299p4LW8yWpRt4Aq1lD8p"
new_agent_schema = []
for obj in agent_schema:
    print(obj)
    print(obj['name'])
    print(obj['description'])

    agent_name = obj['name']
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": f"Create a function schema for {obj['name']} trying to achieve {obj['description']}",
            }
        ]
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=function_schema_agent
    )

    run_status = "in_progress"
    while run_status != "completed":
        ## sleep a second
        time.sleep(2)
        run_status = client.beta.threads.runs.retrieve(run_id=run.id, thread_id=thread.id,).status
        print(f"{agent_name}: {run_status}")

    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages)

    message_str = messages.data[0].content[0].text.value
    print(message_str)

    filename = save_code_from_markdown("function_schema", message_str)
    with open(filename, "r") as file:
        function_definition = json.loads(file.read())

    obj["tools"] = function_definition
    new_agent_schema.append(obj)
    with open("new_agent_schema.json", "w") as file:
        file.write(json.dumps(new_agent_schema))

    print(agent_schema)

In [ ]:
## Loop through each agent in the new schema and create code blocks for each function specified. Save the code blocks to a file in a folder named after the agent
import os
function_code_writer_agent = "asst_jMKQiATQEqBEuRBmaSB6zD6s"
for agent in new_agent_schema:
    agent_name = agent['name']
    os.mkdir(agent_name)
    for function in agent['tools']:
        function_name = function["function"]['name']

        thread = client.beta.threads.create(
        messages=[
                {
                    "role": "user",
                    "content": f"Here is the function schema for {function_name} :\n {json.dumps(function)}",
                }
            ]
        )

        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=function_code_writer_agent
        )

        run_status = "in_progress"
        while run_status != "completed":
            ## sleep a second
            time.sleep(2)
            run_status = client.beta.threads.runs.retrieve(run_id=run.id, thread_id=thread.id,).status
            print(f"{agent_name}: {run_status}")

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(messages)

        message_str = messages.data[0].content[0].text.value
        print(message_str)

        filename = f"{agent_name}/{function_name}.json"
        save_code_from_markdown(filename, message_str)